In [2]:
import itertools
import numpy as np
from math import pi

from qiskit.circuit import QuantumRegister
from qiskit.quantum_info import Statevector
from qiskit import QuantumCircuit, execute, Aer

In [3]:
precision_states = 3
number_states = 4
move_id_len = 2
number_ancillas = 4
number_steps = 5

beta_value = 10

device = Aer.get_backend('statevector_simulator')

In [4]:
def create_initialization_states():

        states = [QuantumRegister(precision_states, name = 'state_' + str(i)) for i in range(number_states)]

        circuit = QuantumCircuit()
        for s in states[::-1]: circuit.add_register(s)

        # initialize superposition
        for s in states: circuit.h(s)

        return circuit.to_gate(label='initialization_states')

In [5]:
def move_preparation():

    move_id = QuantumRegister(move_id_len) 
    move_value = QuantumRegister(1)

    circuit = QuantumCircuit(move_value,move_id)

    # if the successor is generated by sum/substract is necessary to include a bit is superposition 0 or 1
    circuit.h(move_value)
    
    # initialize an uniform superposition with the number of coordinates and append this gate to the circuit
    circuit.h(move_id)
    
    return circuit.to_gate(label='move_preparation')

In [6]:
def gen_oracle(step_beta):

        states = [QuantumRegister(precision_states, name = 'states_' + str(i)) for i in range(number_states)]
        move_id = QuantumRegister(move_id_len) 
        move_value = QuantumRegister(1)
        ancillas = QuantumRegister(number_ancillas, name='ancillas')

        circuit = QuantumCircuit(ancillas, move_value, move_id)
        for s in states[::-1]: circuit.add_register(s)

        # add QRAM to circuit

        return circuit

In [7]:
def coin_flip():
    '''
    Prepares the coin with the probability encoded in the ancilla.
    The important thing to notice is that we are using the same convention as qiskit: littleendian.
    That means that the larger the index of the ancilla bit, the more significant it is, and the larger the rotation
    '''
    # It is necessary to use the saved number in the ancillas in order to perform controlled rotations
    # Notice that ancilla encodes 1-probability, rather than probability.
    # Notice also that cu3(theta) rotates theta/2. As the first coord to rotate is pi/4 we need to start in theta = pi/2

    ancilla = QuantumRegister(number_ancillas, name='ancillas')
    coin = QuantumRegister(1, 'coin')

    circuit = QuantumCircuit(ancilla, coin)

    circuit.x(coin) # Start in 1 and decrease it, since we encoded the coord corresponding 1-probability
    for i in range(ancilla.size-1,-1,-1): # See how to perform an rx rotation in https://qiskit.org/documentation/stubs/qiskit.circuit.library.U3Gate.html
        circuit.cu(theta = -pi*2**(i-ancilla.size), phi  = 0, lam = 0, control_qubit = ancilla[i], target_qubit = coin, gamma = 0)

    return circuit.to_gate(label="coin_flip")

In [8]:
def coin_flip_func(step_beta):
        
    '''
    Defines de coin_flip_gate using the oracle that is provided on the moment.
    '''

    states = [QuantumRegister(precision_states, name = 'states_' + str(i)) for i in range(number_states)]
    move_id = QuantumRegister(move_id_len, name = 'move_id')
    move_value = QuantumRegister(1, name = 'move_value')
    coin = QuantumRegister(1, name = 'coin')
    ancillas = QuantumRegister(number_ancillas, name = 'ancilla')

    circuit = QuantumCircuit(ancillas, coin, move_value, move_id)
    for s in states[::-1]: circuit.add_register(s)

    # Main operations
    circuit.append(gen_oracle(step_beta), move_value[:]+move_id[:]+list(itertools.chain(*states[::-1]))+ancillas[:])
    circuit.append(coin_flip(), ancillas[:]+coin[:])
    circuit.append(gen_oracle(step_beta).inverse(), move_value[:]+move_id[:]+list(itertools.chain(*states[::-1]))+ancillas[:])

    return circuit.to_gate(label="coin_flip_func")

In [9]:
def sum1(state_size):

    qubit_string = QuantumRegister(state_size, "state")
    control = QuantumRegister(1, "control")
    start = QuantumRegister(1, "start")
    end = QuantumRegister(1, "end")

    circuit = QuantumCircuit(qubit_string, control, start, end)
    
    circuit.cx(control,end) # iff control = 1, end = 1
    circuit.x(start)
    circuit.cx(control,start) # iff control = 1, start = 0

    for i in range(qubit_string.size,-1,-1):
        '''
        Next thing we analise if all qubits to the right have value 1, 
        and save it in the current qubit and start.
        Don't need to add control, since end already does that work
        '''
        control_qubits = [c for idx,c in enumerate(qubit_string) if idx!=i][::-1]+end[:]

        if i < qubit_string.size:
            # For i = 0, there is only the start to worry about
            circuit.mcx(control_qubits=control_qubits, target_qubit = qubit_string[i])
        circuit.mcx(control_qubits=control_qubits, target_qubit = start)

        '''
        Next, controlling on the current qubit and start, we change all the following qubits to 0.
        We have to control with qubit_string[n_qubit], start and control because start could be at state 1 without control also being in that state.
        '''
        if i == qubit_string.size:
            for j in range(i-1,-1,-1):
                circuit.ccx(control,start,qubit_string[j])
            circuit.ccx(control,start,end)
        elif i == 0:
            circuit.mcx(control_qubits = [control,qubit_string[i],start], target_qubit = end)
        else:
            for j in range(i-1,-1,-1):            
                circuit.mcx(control_qubits = [control,qubit_string[i],start], target_qubit = qubit_string[j])
            circuit.mcx(control_qubits = [control,qubit_string[i],start], target_qubit = end)
    circuit.x(start)

    return circuit.to_gate(label="sum1")
        
def sumsubtract1(state_size):

    qubit_string = QuantumRegister(state_size, "state")
    control = QuantumRegister(1, "control")
    start = QuantumRegister(1, "start")
    end = QuantumRegister(1, "end")
    move_value = QuantumRegister(1, "move_value_0")

    circuit = QuantumCircuit(qubit_string, control, start, end, move_value)

    circuit.cx(move_value,qubit_string)
    circuit.append(sum1(state_size), qubit_string[:]+control[:]+start[:]+end[:])
    circuit.cx(move_value,qubit_string)

    return circuit.to_gate(label="sumsubtract1")

In [10]:
def sequential_move(circuit, states, move_id, move_value, coin, ancilla):

    # For each coordinate
    for state_index in range(number_states):

        state = states[state_index] #Select the coord from the list of registers
        state_index_bin = np.binary_repr(state_index, width=move_id_len) #convert i to binary

        # Put the given move_id in all 1 to control on it: for instance if we are controling on i=2, move 010 ->111
        for i in range(len(state_index_bin)):
            if state_index_bin[i]=='0':
                circuit.x(move_id[i])

        circuit.mcx(control_qubits=[coin[0]]+[move_id[i] for i in range(move_id.size)], target_qubit = ancilla[0])#create a single control
        circuit.append(sumsubtract1(len(state)), state[:]+[ancilla[0]]+[ancilla[1]]+[ancilla[2]]+[move_value[0]])
        circuit.mcx(control_qubits= [coin[0]]+[move_id[i] for i in range(move_id.size)], target_qubit = ancilla[0])#create a single control 

        # Undo the move_id preparation: for instance, if we are controlling on i= 2 move 111->010
        for i in range(len(state_index_bin)):
            if state_index_bin[i]=='0':
                circuit.x(move_id[i])

    return circuit.to_gate(label='sequential_move')


In [11]:
def conditional_move_coord():

    move_id = QuantumRegister(move_id_len, name='move_id')
    move_value = QuantumRegister(1, name='move_value')
    coin = QuantumRegister(1, name='coin')
    ancilla = QuantumRegister(number_ancillas, name='ancillas')
    states = [QuantumRegister(precision_states, name = 'coord_' + str(i)) for i in range(number_states)]

    circuit = QuantumCircuit(ancilla, coin, move_value, move_id)
    for s in states[::-1]: circuit.add_register(s)

    return sequential_move(circuit, states, move_id, move_value, coin, ancilla)

In [12]:
def reflection():

    move_id = QuantumRegister(move_id_len) 
    move_value = QuantumRegister(1)
    coin = QuantumRegister(1)

    circuit = QuantumCircuit(coin, move_value, move_id)

    circuit.x(move_id)
    circuit.x(move_value)
    circuit.x(coin)
    
    # Perform a multicontrolled Z
    circuit.h(coin[0])
    circuit.mcx(control_qubits=move_id[:]+move_value[:], target_qubit = coin[0])
    circuit.h(coin[0])
    
    circuit.x(move_id)
    circuit.x(move_value)
    circuit.x(coin)

    return circuit.to_gate(label='reflection')

In [13]:
def W_func(step_beta):
        
        '''This defines the parametrised gate W using the oracle that is provided to it, and we can reuse its inverse too.'''
        ''' W = RV^B^FBV'''

        states = [QuantumRegister(precision_states, name = 'state_' + str(i)) for i in range(number_states)]
        move_id = QuantumRegister(move_id_len, name = 'move_id')
        move_value = QuantumRegister(1, name = 'move_value')
        coin = QuantumRegister(1, name = 'coin')
        ancillas = QuantumRegister(number_ancillas, name = 'ancilla')

        circuit = QuantumCircuit(ancillas, coin, move_value, move_id)
        for s in states[::-1]: circuit.add_register(s)

        # Move preparation V
        circuit.append(move_preparation(), move_value[:]+move_id[:])

        # Coin flip B
        circuit.append(coin_flip_func(step_beta), ancillas[:]+coin[:]+move_value[:]+move_id[:]+list(itertools.chain(*states[::-1])))

        # Conditional move F
        circuit.append(conditional_move_coord(), ancillas[:]+coin[:]+move_value[:]+move_id[:]+list(itertools.chain(*states[::-1])))

        # Inverse coin flip B^
        circuit.append(coin_flip_func(step_beta).inverse(),ancillas[:]+coin[:]+move_value[:]+move_id[:]+list(itertools.chain(*states[::-1])))

        # Inverse move preparation V^
        circuit.append(move_preparation().inverse(), move_value[:]+move_id[:])

        # Reflection R
        circuit.append(reflection(), coin[:]+move_value[:]+move_id[:])

        return circuit.to_gate(label="w")

In [14]:
# State definition. All coords range from 0 to 2pi
states = [QuantumRegister(precision_states, name = 'state_' + str(i)) for i in range(number_states)]
move_id = QuantumRegister(move_id_len, name = 'move_id')
move_value = QuantumRegister(1, name = 'move_value')
coin = QuantumRegister(1, name = 'coin')
ancillas = QuantumRegister(number_ancillas, name = 'ancilla')

circuit = QuantumCircuit(ancillas, coin, move_value, move_id)
for s in states[::-1]: circuit.add_register(s)

# initializate all states to create a superposition and start applying W to this state
circuit.append(create_initialization_states(), list(itertools.chain(*states[::-1])))

for step_beta in range(2, number_steps+1):
                
    circuit.append(W_func(step_beta), ancillas[:]+coin[:]+move_value[:]+move_id[:]+list(itertools.chain(*states[::-1])))

job = execute(circuit, device)

counts = job.result().get_counts()

QiskitError: 'One or more instructions cannot be converted to a gate. "circuit-97" is not a gate instruction'